In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
import time
import os
import warnings
warnings.filterwarnings('ignore')

# Models
from sklearn.ensemble import (RandomForestClassifier, GradientBoostingClassifier, 
                             ExtraTreesClassifier)
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# ==========================================
# CONFIGURATION
# ==========================================

FEAT_DIR = r"C:\Users\htagi\Downloads\Back\features"
OUTPUT_DIR = r"C:\Users\htagi\Downloads\Back\experiments_results"
os.makedirs(OUTPUT_DIR, exist_ok=True)
RANDOM_STATE = 42

print("="*70)
print(" BACTERIA CLASSIFICATION - EXPERIMENTS")
print("="*70)

# ==========================================
# LOAD DATA
# ==========================================
print("\n Loading data...")
X_train = np.load(os.path.join(FEAT_DIR, "features_train.npy"))
y_train_raw = np.load(os.path.join(FEAT_DIR, "labels_train.npy"))
X_val = np.load(os.path.join(FEAT_DIR, "features_val.npy"))
y_val_raw = np.load(os.path.join(FEAT_DIR, "labels_val.npy"))

# Encode labels
le = LabelEncoder()
y_train = le.fit_transform(y_train_raw)
y_val = le.transform(y_val_raw)

print(f"Training set: {X_train.shape}")
print(f"Validation set: {X_val.shape}")
print(f"Classes: {le.classes_}")

# CV setup
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

# ==========================================
# EXPERIMENT 1: MODEL COMPARISON
# ==========================================
print("\n" + "="*70)
print(" EXPERIMENT 1: Comparing Different Models (with SMOTE)")
print("="*70)

models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=RANDOM_STATE),
    'Naive Bayes': GaussianNB(),
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'SVM': SVC(kernel='rbf', probability=True, random_state=RANDOM_STATE),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1),
    'Extra Trees': ExtraTreesClassifier(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=RANDOM_STATE),
    'XGBoost': XGBClassifier(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1, verbosity=0)
}

results = []

for model_name, model in models.items():
    print(f"\nTesting {model_name}...")
    start = time.time()
    
    # Create pipeline
    if model_name in ['Logistic Regression', 'SVM', 'KNN']:
        pipeline = ImbPipeline([
            ('scaler', StandardScaler()),
            ('smote', SMOTE(random_state=RANDOM_STATE, k_neighbors=3)),
            ('clf', model)
        ])
    else:
        pipeline = ImbPipeline([
            ('smote', SMOTE(random_state=RANDOM_STATE, k_neighbors=3)),
            ('clf', model)
        ])
    
    # Cross-validation
    scores = []
    for fold, (tr_idx, val_idx) in enumerate(cv.split(X_train, y_train), 1):
        X_tr, X_v = X_train[tr_idx], X_train[val_idx]
        y_tr, y_v = y_train[tr_idx], y_train[val_idx]
        
        pipeline.fit(X_tr, y_tr)
        y_pred = pipeline.predict(X_v)
        f1 = f1_score(y_v, y_pred, average='macro')
        scores.append(f1)
    
    mean_f1 = np.mean(scores)
    std_f1 = np.std(scores)
    elapsed = time.time() - start
    
    results.append({
        'Model': model_name,
        'F1-Score': mean_f1,
        'Std': std_f1,
        'Time (s)': elapsed
    })
    
    print(f"  F1-macro: {mean_f1:.4f} (±{std_f1:.4f})")
    print(f"  Time: {elapsed:.1f}s")

# Results DataFrame
df_results = pd.DataFrame(results).sort_values('F1-Score', ascending=False)
print("\n" + "="*70)
print("RESULTS SUMMARY:")
print("="*70)
print(df_results.to_string(index=False))
print("="*70)

# Save results
df_results.to_csv(os.path.join(OUTPUT_DIR, 'experiment1_model_comparison.csv'), index=False)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

colors = ['green' if i == 0 else 'steelblue' for i in range(len(df_results))]
axes[0].barh(df_results['Model'], df_results['F1-Score'], color=colors, edgecolor='black')
axes[0].set_xlabel('F1-Score (macro)', fontsize=12, fontweight='bold')
axes[0].set_title('Model Performance Comparison', fontsize=14, fontweight='bold')
axes[0].grid(axis='x', alpha=0.3)
axes[0].set_xlim(0, 1)

for i, (model, score) in enumerate(zip(df_results['Model'], df_results['F1-Score'])):
    axes[0].text(score + 0.01, i, f'{score:.4f}', va='center', fontsize=10, fontweight='bold')

axes[1].barh(df_results['Model'], df_results['Time (s)'], color='coral', edgecolor='black')
axes[1].set_xlabel('Training Time (seconds)', fontsize=12, fontweight='bold')
axes[1].set_title('Model Training Time', fontsize=14, fontweight='bold')
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'experiment1_comparison.png'), dpi=300, bbox_inches='tight')
print(f"Saved: experiment1_comparison.png")
plt.close()

best_model_name = df_results.iloc[0]['Model']
best_f1 = df_results.iloc[0]['F1-Score']
print(f"\n Best Model: {best_model_name} with F1={best_f1:.4f}")

# ==========================================
# EXPERIMENT 2: FEATURE SELECTION
# ==========================================
print("\n" + "="*70)
print("EXPERIMENT 2: Feature Selection Impact")
print("="*70)

feature_counts = [10, 20, 30, 40, 50, 61]
selection_results = []

for n_features in feature_counts:
    print(f"\nTesting with {n_features} features...")
    
    if n_features < 61:
        selector = SelectKBest(score_func=f_classif, k=n_features)
        X_selected = selector.fit_transform(X_train, y_train)
    else:
        X_selected = X_train
    
    pipeline = ImbPipeline([
        ('scaler', StandardScaler()),
        ('smote', SMOTE(random_state=RANDOM_STATE, k_neighbors=3)),
        ('clf', XGBClassifier(n_estimators=100, random_state=RANDOM_STATE, n_jobs=-1, verbosity=0))
    ])
    
    scores = []
    for tr_idx, val_idx in cv.split(X_selected, y_train):
        X_tr, X_v = X_selected[tr_idx], X_selected[val_idx]
        y_tr, y_v = y_train[tr_idx], y_train[val_idx]
        
        pipeline.fit(X_tr, y_tr)
        y_pred = pipeline.predict(X_v)
        scores.append(f1_score(y_v, y_pred, average='macro'))
    
    mean_f1 = np.mean(scores)
    selection_results.append({'N_Features': n_features, 'F1-Score': mean_f1})
    print(f"  F1-macro: {mean_f1:.4f}")

df_selection = pd.DataFrame(selection_results)
print("\n" + "="*70)
print(df_selection.to_string(index=False))
print("="*70)

# Save results
df_selection.to_csv(os.path.join(OUTPUT_DIR, 'experiment2_feature_selection.csv'), index=False)

# Visualize
plt.figure(figsize=(10, 6))
plt.plot(df_selection['N_Features'], df_selection['F1-Score'], 
         marker='o', linewidth=2, markersize=10, color='steelblue')
plt.xlabel('Number of Features', fontsize=12, fontweight='bold')
plt.ylabel('F1-Score (macro)', fontsize=12, fontweight='bold')
plt.title('Feature Selection Impact on Model Performance', fontsize=14, fontweight='bold')
plt.grid(alpha=0.3)
plt.xticks(df_selection['N_Features'])

best_idx = df_selection['F1-Score'].idxmax()
best_n = df_selection.loc[best_idx, 'N_Features']
best_score = df_selection.loc[best_idx, 'F1-Score']
plt.scatter([best_n], [best_score], color='red', s=200, zorder=5, 
           marker='*', edgecolors='black', linewidth=2)
plt.text(best_n, best_score + 0.01, f'Best: {best_n} features\nF1={best_score:.4f}',
        ha='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'experiment2_feature_selection.png'), dpi=300, bbox_inches='tight')
print(f"Saved: experiment2_feature_selection.png")
plt.close()

print(f"\n Conclusion: Using {best_n} features gives best F1-Score of {best_score:.4f}")

# ==========================================
# EXPERIMENT 3: HYPERPARAMETER TUNING
# ==========================================
print("\n" + "="*70)


 BACTERIA CLASSIFICATION - EXPERIMENTS

 Loading data...
Training set: (11608, 61)
Validation set: (2901, 61)
Classes: ['Basophil' 'Eosinophil' 'Lymphocyte' 'Monocyte' 'Neutrophil']

🔬 EXPERIMENT 1: Comparing Different Models (with SMOTE)

Testing Logistic Regression...
  F1-macro: 0.7558 (±0.0092)
  Time: 23.0s

Testing Naive Bayes...
  F1-macro: 0.3822 (±0.0107)
  Time: 0.6s

Testing KNN...
  F1-macro: 0.6407 (±0.0066)
  Time: 1.7s

Testing SVM...
  F1-macro: 0.8381 (±0.0061)
  Time: 653.7s

Testing Random Forest...
  F1-macro: 0.8099 (±0.0087)
  Time: 49.1s

Testing Extra Trees...
  F1-macro: 0.7890 (±0.0055)
  Time: 11.0s

Testing Gradient Boosting...
  F1-macro: 0.8108 (±0.0116)
  Time: 40712.1s

Testing XGBoost...
  F1-macro: 0.8516 (±0.0073)
  Time: 42.6s

RESULTS SUMMARY:
              Model  F1-Score      Std     Time (s)
            XGBoost  0.851553 0.007252    42.631077
                SVM  0.838123 0.006122   653.721462
  Gradient Boosting  0.810799 0.011571 40712.083271
 